# Tracy Giskard Demo

## 1. Launch Tracy simulation

In [ ]:
import rospy
from utils import open_desktop, run_command

In [ ]:
# Open a desktop GUI left sidecar
# Note: if you see connection error, try running this code cell again
open_desktop()

In [ ]:
# Run the roslaunch in a background process
# Note: To see the logs, run this command in a terminal tab.
run_command('roslaunch giskardpy giskardpy_tracy_standalone.launch')

## 2. Control the Tracy with the Giskard Python Interface

In [ ]:
# initialize a ros node and the python wrapper for giskard
import rospy
from pprint import pprint
from giskardpy.python_interface import GiskardWrapper
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped
rospy.init_node('gk_python_demo')
giskard_wrapper = GiskardWrapper()

### List Joints and links

In [ ]:
# List all the controlled joints
robot_joints = giskard_wrapper.get_controlled_joints(giskard_wrapper.get_group_names()[0])
# List all the links
robot_links = giskard_wrapper.get_group_info(giskard_wrapper.get_group_names()[0]).links

print('Joints:')
pprint(robot_joints)
print('Links:')
pprint(robot_links)

### Define a default pose

In [ ]:
default_pose = {
        'left_shoulder_pan_joint': 2.539670467376709,
        'left_shoulder_lift_joint': -1.46823854119096,
        'left_elbow_joint': 2.1197431723224085,
        'left_wrist_1_joint': -1.4825000625899811,
        'left_wrist_2_joint': 5.467689037322998,
        'left_wrist_3_joint': -0.9808381239520472,
        'right_shoulder_pan_joint': 3.7588136196136475,
        'right_shoulder_lift_joint': -1.7489210567870082,
        'right_elbow_joint': -2.054229259490967,
        'right_wrist_1_joint': -1.6140786610045375,
        'right_wrist_2_joint': 0.7295855283737183,
        'right_wrist_3_joint': 3.944669485092163,
    }
# move to default pose
def reset_to_default_pose():
    giskard_wrapper.set_joint_goal(default_pose)
    giskard_wrapper.plan_and_execute()

reset_to_default_pose()

### Spawn and attach cylinders to the hands

In [ ]:
cylinder_name_r = 'C_r'
cylinder_height = 0.121
pose = PoseStamped()
pose.header.frame_id = 'r_gripper_tool_frame'
pose.pose.position.z = cylinder_height / 5
pose.pose.orientation.w = 1
giskard_wrapper.add_cylinder(name=cylinder_name_r,
                         height=cylinder_height,
                         radius=0.0225,
                         pose=pose,
                         parent_link='r_gripper_tool_frame')
# dye it blue
giskard_wrapper.dye_group(cylinder_name_r, (0.5, 1, 1, 1))

cylinder_name_l = 'C_l'
cylinder_height = 0.121
pose = PoseStamped()
pose.header.frame_id = 'l_gripper_tool_frame'
pose.pose.position.z = cylinder_height / 5
pose.pose.orientation.w = 1
giskard_wrapper.add_cylinder(name=cylinder_name_l,
                         height=cylinder_height,
                         radius=0.0225,
                         pose=pose,
                         parent_link='l_gripper_tool_frame')
# dye it yellow
giskard_wrapper.dye_group(cylinder_name_l, (1, 1, 0, 1))

### Call a goal to "insert" the cylinder at a point

In [ ]:
def insert_cylider_to(x, y, cylinder_name):
    hole_point = PointStamped()
    hole_point.header.frame_id = 'table'
    hole_point.point.x = x
    hole_point.point.y = y
    giskard_wrapper.set_json_goal('InsertCylinder',
                              cylinder_name=cylinder_name,
                              cylinder_height=cylinder_height,
                              hole_point=hole_point)
    giskard_wrapper.allow_all_collisions()
    giskard_wrapper.plan_and_execute()

### Insert cylinders to 10 random positions

In [ ]:
import numpy as np

num = 10  # Number of coordinate pairs to generate
x_min, x_max = 0.5, 0.9  # Range for x coordinates
y_min, y_max = 0.2, 0.7  # Range for y coordinates

coordinates_l = np.column_stack((np.random.uniform(x_min, x_max, num), np.random.uniform(y_min, y_max, num)))
coordinates_r = np.column_stack((np.random.uniform(x_min, x_max, num), np.random.uniform(y_min, y_max, num)))

for i in range(num):  
    insert_cylider_to(coordinates_r[i][0], -coordinates_r[i][1], cylinder_name_r)
    insert_cylider_to(coordinates_l[i][0], coordinates_l[i][1], cylinder_name_l)
    reset_to_default_pose()